In [16]:
import glob
import json
import pandas as pd
import numpy as np
from collections import defaultdict
from datetime import datetime
from dateutil import tz

PATH_ELECTRICITY='dataset/electricity_data_all/*'
PATH_WEATHER='dataset/melbourne-weather/melbourne-weather.json'
PATH_WEATHER2='dataset/melbourne-weather/melbourne-2020.csv'
PATH_NBN='dataset/nbn.csv'
#Index(['REGION', 'SETTLEMENTDATE', 'TOTALDEMAND', 'RRP', 'PERIODTYPE'], dtype='object')

In [17]:
# READ ELECTRICITY

day2rrp = defaultdict(list)
day2demand = defaultdict(list)

def get_day_month(day_str):
    day = day_str.split()[0]
    month = '/'.join(day.split('/')[:2])
    return day, month

for file in glob.glob(PATH_ELECTRICITY):
    df = pd.read_csv(file)
    for idx, row in df.iterrows():
        day, month = get_day_month(row['SETTLEMENTDATE'])        
        day2demand[day].append(row['TOTALDEMAND'])
        day2rrp[day].append(row['RRP'])    

# Daily Average
for day in day2rrp.keys():
    day2rrp[day] = np.sum(day2rrp[day])
    day2demand[day] = np.sum(day2demand[day])

In [18]:
# READ WHEATHER

def is_holiday(time):
    # month/day
    holidays = ['01/01', '01/27', '03/09', '04/10', '04/25', '06/08', \
     '10/23', '12/25', '12/26', '12/27', '12/28', '12/29', '12/30', '12/31']
    return int(time.strftime("%m/%d") in holidays or time.strftime('%A') in ['Saturday', 'Sunday'])# or time.strftime("%Y/%m") > '2020/04/00')
    
def read_date(str_date):
    from_zone = tz.gettz('UTC')
    to_zone = tz.gettz('Australia/Melbourne')
    utc = datetime.strptime(str_date, "%Y-%m-%d %H:%M:%S +0000 UTC")
    utc = utc.replace(tzinfo=from_zone)
    # Convert time zone
    aest = utc.astimezone(to_zone)
    return aest.strftime("%Y/%m/%d"), is_holiday(aest)

class Weather:
    def __init__(self, dic):
        self.temp_min = dic['main']['temp_min']
        self.temp_max = dic['main']['temp_max']
        self.feels_like = dic['main']['feels_like']
        self.pressure = dic['main']['pressure']
        self.humidity = dic['main']['humidity']
        self.wind_speed = dic['wind']['speed']
        self.wind_deg = dic['wind']['deg']       
        
day2weather = defaultdict(list)
day2offDay = defaultdict(int)
data = json.load(open(PATH_WEATHER))
for datum in data:
    day, is_offday = read_date(datum['dt_iso'])
    day2offDay[day] = is_offday
    day2weather[day].append(Weather(datum))
    
data2020 = pd.read_csv(PATH_WEATHER2)
for idx, row in data2020.iterrows():
    day, is_offday = read_date(row['dt_iso'])
    day2offDay[day] = is_offday
    dic = {'main': {'temp_min': row['temp_min'], 'temp_max': row['temp_max'], 'feels_like': row['feels_like'],
                   'pressure': row['pressure'], 'humidity': row['humidity']},
          'wind': {'speed': row['wind_speed'], 'deg': row['wind_deg']}}
    day2weather[day].append(Weather(dic))

In [19]:
# READ BROADBAND

month2broadband = {}
data = pd.read_csv(PATH_NBN)
for idx, row in data.iterrows():
    month2broadband[row['month']] = row

In [20]:
def get_average(weather_list):
    temp_mins=[];temp_maxs=[];feels_likes=[];pressures=[]
    humidities=[];wind_speeds=[];wind_degs=[]
    for w in weather_list:
        temp_mins.append(w.temp_min)
        temp_maxs.append(w.temp_max)
        feels_likes.append(w.feels_like)
        pressures.append(w.pressure)
        humidities.append(w.humidity)
        wind_speeds.append(w.wind_speed)
        wind_degs.append(w.wind_deg)
    return np.mean(temp_mins), np.mean(temp_maxs), np.mean(feels_likes), np.mean(pressures), \
        np.mean(humidities), np.mean(wind_speeds), np.mean(wind_degs)

In [21]:
cols = ['day', 'temp_min', 'temp_max', 'feels_like', 'pressure', 'humidity', 
        'wind_speed', 'wind_deg', 'is_offday',# 'ready-to-connect',
        #'connected', 'first-time', 'agreed-first-time', 
        #'average-congestion', 'fix-line', 
        'whole-sale-1', 'whole-sale-2', 
        #'avail', 'nbn-fault','fault-per-100',
        'demand', 'rrp']
df = pd.DataFrame(columns=cols)

for day in day2weather.keys():
    avgs = get_average(day2weather[day])
    month = day.split('/')
    month = month[1]+'/'+month[0]
    df = df.append({
        'day':day,
        'temp_min':avgs[0],
        'temp_max':avgs[1],
        'feels_like': avgs[2],
        'pressure': avgs[3],
        'humidity': avgs[4],
        'wind_speed': avgs[5],
        'wind_deg': avgs[6],
        'is_offday': day2offDay[day],
        #'ready-to-connect': month2broadband[month]['ready-to-connect'],
        #'connected': month2broadband[month]['connected'],
        #'first-time': month2broadband[month]['first-time'],
        #'agreed-first-time': month2broadband[month]['agreed-first-time'],
        #'average-congestion': month2broadband[month]['average-congestion'],
        #'fix-line': month2broadband[month]['fix-line'],
        'whole-sale-1': month2broadband[month]['whole-sale-1'],
        'whole-sale-2': month2broadband[month]['whole-sale-2'],
        #'avail': month2broadband[month]['avail'],
        #'nbn-fault': month2broadband[month]['nbn-fault'],
        #'fault-per-100': month2broadband[month]['fault-per-100'],
        'demand': day2demand[day],
        'rrp': day2rrp[day]
    }, ignore_index=True)

In [22]:
df

,day,temp_min,temp_max,feels_like,pressure,humidity,wind_speed,wind_deg,is_offday,whole-sale-1,whole-sale-2,demand,rrp
0,2015/01/01,18.818462,23.249231,19.723846,1014.769231,56.307692,2.880769,166.461538,1,16,84,199270.06,1173.47
1,2015/01/02,23.112500,28.815833,23.994583,1012.041667,44.833333,3.297083,127.583333,0,16,84,259212.02,1485.13
2,2015/01/03,27.044583,33.135000,25.151667,1009.500000,30.958333,7.962917,76.875000,1,16,84,284601.08,1590.18
3,2015/01/04,17.069583,21.049583,16.899167,1016.208333,62.166667,4.544583,189.375000,1,16,84,208661.43,1188.25
4,2015/01/05,15.143333,19.896667,15.582083,1020.166667,61.708333,3.671250,184.208333,0,16,84,236264.40,1233.09
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2096,2020/09/27,7.632917,11.917917,5.662083,1024.416667,72.250000,4.370833,203.125000,1,69,31,203406.98,2546.82
2097,2020/09/28,7.253750,11.949583,6.875417,1027.041667,74.416667,2.401667,187.166667,0,69,31,229302.28,2704.58
2098,2020/09/29,10.448333,14.254167,8.205000,1020.958333,66.416667,4.604167,219.583333,0,69,31,224152.92,1953.21
2099,2020/09/30,12.799600,16.258800,10.769200,1010.400000,71.920000,5.688000,308.800000,0,69,31,227240.42,1786.40


In [23]:
df.to_csv('preprocessed/daily_data_with_auxiliary.csv', index=False)